# Why right multiplication in GCN : AXW ?



In graph neural networks (GNNs), the standard formulation involves using right matrix multiplication, i.e., multiplying the adjacency matrix (A) with the node feature matrix (X) on the right side. This formulation allows the nodes to aggregate information from their neighbors and capture the local structural information in the graph.

To understand why this formulation is necessary, let's consider a simple graph with three nodes and their corresponding features:

In [3]:
# Node 1: [1, 2]
# Node 2: [3, 4]
# Node 3: [5, 6]

In [4]:
A = [[0, 1, 0],
     [1, 0, 1],
     [0, 1, 0]]

The entry A[i, j] represents the weight of the edge between nodes i and j. In this example, node 1 is connected to node 2, node 2 is connected to both node 1 and node 3, and node 3 is connected to node 2.

Let's calculate the result of right matrix multiplication (AX) step by step using PyTorch:

In [5]:
import torch

# Node feature matrix (X)
X = torch.tensor([[1, 2],
                  [3, 4],
                  [5, 6]])

# Adjacency matrix (A)
A = torch.tensor([[0, 1, 0],
                  [1, 0, 1],
                  [0, 1, 0]])

# Right matrix multiplication: AX
AX = torch.matmul(A, X)
print("AX:")
print(AX)


AX:
tensor([[3, 4],
        [6, 8],
        [3, 4]])


Therefore, by performing right matrix multiplication (AX), we effectively aggregate the features of neighboring nodes for each node in the graph, capturing the local information.


Let's calculate the result of the GNN update rule (AXW) step by step using PyTorch:

In [13]:
import torch
import torch.nn as nn

# Node feature matrix (X)
X = torch.tensor([[1., 2.],
                  [3., 4.],
                  [5., 6.]])

# Adjacency matrix (A)
A = torch.tensor([[0., 1., 0.],
                  [1., 0., 1.],
                  [0., 1., 0.]])

# Weight matrix (W)
W = nn.Parameter(torch.randn(2, 2))

# GNN update: AXW




AXW = torch.matmul(torch.matmul(A, X), W)
output = torch.sigmoid(AXW)
print("Output:")
print(output)





Output:
tensor([[0.6109, 0.2292],
        [0.7114, 0.0812],
        [0.6109, 0.2292]], grad_fn=<SigmoidBackward0>)


In this example, we introduced the weight matrix W as an nn.Parameter with dimensions [2, 2]. We performed the GNN update rule by multiplying A and X using right matrix multiplication (torch.matmul(A, X)), followed by multiplying the result with W (torch.matmul(torch.matmul(A, X), W)). Finally, we applied the sigmoid activation function to the result.

The inclusion of the weight matrix W allows the GNN to learn and transform the aggregated features according to the task at hand. The choice of activation function and the specific architecture of the GNN can vary depending on the requirements of the problem. The example provided is a basic illustration to demonstrate the inclusion of the weight matrix W in the GNN formulation.

# What goes wrong if we had AWX

In the alternative formulation, AWX, we perform element-wise multiplication between the adjacency matrix A and the node feature matrix X, followed by the matrix multiplication with X. However, this formulation leads to a mismatch in dimensions and fails to capture the desired neighborhood aggregation.



Specifically, in the standard formulation, the adjacency matrix A represents the connections between nodes in a graph. Each entry A[i, j] represents the weight of the edge between nodes i and j. The node feature matrix X contains the feature vectors of each node, where each row corresponds to a node and each column corresponds to a feature dimension.

By multiplying A and X using right matrix multiplication (AX), we effectively perform a weighted sum of the neighboring nodes' features for each node in the graph. This operation allows the node to aggregate information from its neighbors, capturing the local structural information in the graph. The resulting matrix AX has dimensions (N x F), where N is the number of nodes in the graph and F is the number of features.

If we were to use the alternative formulation AWX, the multiplication would be done differently. Here, AW would represent a weighted adjacency matrix, where each entry AW[i, j] would denote the contribution of node j's features to node i. However, the subsequent multiplication with X would cause a mismatch in dimensions.


In [18]:
import torch

# Node feature matrix (X)
X = torch.tensor([[1., 2.],
                  [3., 4.]])

# Adjacency matrix (A)
A = torch.tensor([[0., 1.],
                  [1., 0.]])

# Weight matrix (W)
W = torch.tensor([[0.1, 0.2],
                  [0.3, 0.4],
                  [0.5, 0.6]])


In [27]:
torch.matmul(W,X)

tensor([[0.7000, 1.0000],
        [1.5000, 2.2000],
        [2.3000, 3.4000]])

In [28]:
# But we cannot do
torch.matmul(A,torch.matmul(W,X))

RuntimeError: ignored

In this example, the dimensions of A are [2 x 2], the dimensions of W are [3 x 2], and the dimensions of X are [2 x 2]. When we attempt to compute AWX using the incorrect order, a dimension mismatch occurs. The error message indicates that the matrix multiplication between [2 x 2] and [2 x 3] is not valid because the number of columns in the first matrix does not match the number of rows in the second matrix.

This demonstrates the importance of following the correct order (AXW) in GNN formulations, where the adjacency matrix A is multiplied with the node feature matrix X first, followed by multiplication with the weight matrix W. The correct formulation ensures that the dimensions align correctly for matrix multiplication and preserves the desired neighborhood aggregation.